In [65]:
import json

In [66]:
with open('tagi_gier_wspoldzielone.json', encoding='utf-8') as f:
    data = json.load(f)

In [67]:
label_count={}
for game in data:
  label=game['label']
  if label not in label_count:
    label_count[label]=0
  label_count[label]+= 1

In [68]:
print('Liczba elementów w słowniku: ', len(data))
print('Liczba unikalnych gier', len(label_count))
print('Liczba rekordów dla każdej gry')
for label in label_count:
  print(f'-{label}: {label_count[label]}')

Liczba elementów w słowniku:  300
Liczba unikalnych gier 5
Liczba rekordów dla każdej gry
-minecraft: 60
-fortnite: 60
-roblox: 60
-lol: 60
-cs: 60


In [69]:
tag_count={}
tag_to_labels={}

for game in data:
  label= game['label']
  for tag in game['tags']:
    if tag not in tag_count:
      tag_count[tag]=0
    tag_count[tag]+=1

    if tag not in tag_to_labels:
      tag_to_labels[tag]=[]

    if label not in tag_to_labels[tag]:
      tag_to_labels[tag].append(label)


In [70]:
only_one= 0
multiple_labels=0
for tag in tag_to_labels:
  label_total= len(tag_to_labels[tag])
  if label_total == 1:
    only_one+=1
  else:
    multiple_labels+=1

In [71]:
print('Liczba tagów: ',len(tag_count))
print('Tagi przypisane do jednej gry: ',only_one)
print('Tagi przypisane dla wielu gier: ', multiple_labels)


Liczba tagów:  74
Tagi przypisane do jednej gry:  25
Tagi przypisane dla wielu gier:  49


In [72]:
print("\n20 najczęstszych tagów:")
sorted_tags = sorted(tag_count.items(), key=lambda x: x[1])
for tag, count in sorted_tags[-20:]:
    print(" -", tag, ":", count, "razy")


20 najczęstszych tagów:
 - przetrwanie : 38 razy
 - sklep : 38 razy
 - questy : 39 razy
 - fortyfikacje : 39 razy
 - symulator : 39 razy
 - aktualizacja : 39 razy
 - pistolet : 40 razy
 - waluta : 40 razy
 - karabin : 41 razy
 - arena : 41 razy
 - kopanie : 43 razy
 - oddziały : 43 razy
 - loot : 44 razy
 - rankedy : 44 razy
 - klocki : 46 razy
 - sezon : 46 razy
 - emotki : 46 razy
 - budowanie : 48 razy
 - crafting : 49 razy
 - roleplay : 50 razy


In [73]:
import random, math

In [74]:
def load_data(path='tagi_gier_wspoldzielone.json'):
  with open(path, encoding='utf-8') as f:
    data = json.load(f)
  return data

In [75]:
def train_test_split(data, test_size=0.2):
  random.shuffle(data)
  cut= int(len(data)*(1-test_size))
  print(f'Ilość próbek zbioru uczącego : {len(data[:cut])}')
  print(f'Ilość próbek zbioru testowego: {len(data[cut:])}')
  return data[:cut], data[cut:]

In [76]:
def build_vocabulary(train):
  vocab=set()
  for rec in train:
    vocab.update(rec['tags'])
  print(vocab)
  return vocab

In [77]:
def train_nb(train, vocab, alpha=1.0):
  class_counts={}
  world_counts={}
  total_worlds={}

  for rec in train:
    c= rec['label']
    class_counts[c]= class_counts.get(c, 0) + 1
    world_counts.setdefault(c, {})
    total_worlds.setdefault(c, 0)

    for tag in rec['tags']:
      world_counts[c][tag]= world_counts[c].get(tag, 0) + 1
      total_worlds[c]+=1

  model={
      'class_counts': class_counts,
      'world_counts': world_counts,
      'total_worlds': total_worlds,
      'vocab': vocab,
      'alpha': alpha,
      'total_docs': len(train)
  }
  print(model)
  return model




In [78]:
def log_prob(model, rec, class_name):
  logp= math.log(model['class_counts'][class_name]/ model['total_docs'])
  V= len(model['vocab'])
  a=model['alpha']
  for tag in rec['tags']:
    wc= model['world_counts'][class_name].get(tag, 0)
    logp += math.log((wc +a)/ (model['total_worlds'][class_name] +a *V))

  return logp

In [79]:
def predict(model, rec):
  best_class, best_log = None, -1e99
  for c in model["class_counts"]:
    lp = log_prob(model, rec, c)
    if lp > best_log:
      best_class, best_log = c, lp
    return best_class

In [80]:
def main():
  data=load_data()
  train, test= train_test_split(data)
  vocab=build_vocabulary(train)
  model= train_nb(train, vocab)

  sample=test[0]
  print('\n\nprzykładowe tagi: ', sample['tags'])
  print('Rzeczywista gra: ', sample['label'])
  print('Przewidziana gra: ', predict(model, sample))

main()

Ilość próbek zbioru uczącego : 240
Ilość próbek zbioru testowego: 60
{'drużyna', 'redstone', 'budowanie', 'kolekcje', 'awp', 'battle', 'emotki', 'sandbox', 'nether', 'studio', 'online', 'bomba', 'rankingi', 'strzelanie', 'smok', 'burza', 'strzał w głowę', 'ulti', 'roleplay', 'survival', 'lama', 'crafting', 'prywatny serwer', 'tryb', 'parkour', 'multiplayer', 'wydarzenia', 'karabin', 'sklep', 'czat', 'aktualizacja', 'runy', 'nexus', 'mapa', 'arena', 'tickrate', 'flash', 'dust2', 'v-bucksy', 'autobus bojowy', 'klocki', 'symulator', 'hardcore', 'slurp', 'sezon', 'oddziały', 'skiny', 'fortyfikacje', 'esport', 'diamenty', 'robuxy', 'kopanie', 'waluta', 'loot', 'lua', 'questy', 'baron', 'misje', 'biomy', 'bohaterowie', 'snajper', 'przetrwanie', 'wyzwania', 'pistolet', 'rankedy', 'creepery', 'tryb solo', 'dżungla', 'zombie', 'obby', 'duety', 'brokhaven', 'event', 'tryb kreatywny'}
{'class_counts': {'roblox': 50, 'fortnite': 49, 'lol': 44, 'cs': 52, 'minecraft': 45}, 'world_counts': {'roblox':